In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
x = []
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        x.append(str(os.path.join(dirname)))

print(set(x))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# --- Cell 1: Imports and Setup ---
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Set a style for plots
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

# Define the base input path
BASE_PATH = Path('/kaggle/input/MABe-mouse-behavior-detection/')

In [ ]:
# --- Cell 2: Load and Inspect Metadata ---
train_meta_df = pd.read_csv(BASE_PATH / 'train.csv')

print("Shape of the training metadata:")
print(train_meta_df.shape)

print("\nFirst 5 rows:")
display(train_meta_df.head())

print("\nData types and missing values:")
train_meta_df.info()

In [ ]:
# --- Cell 3: Analyze Lab and Behavior Diversity ---
print("--- Lab Distribution ---")
lab_counts = train_meta_df['lab_id'].value_counts()
print(lab_counts)

plt.figure(figsize=(12, 7))
sns.barplot(x=lab_counts.index, y=lab_counts.values, palette='viridis')
plt.title('Number of Videos per Lab')
plt.xlabel('Lab ID')
plt.ylabel('Video Count')
plt.xticks(rotation=45, ha='right')
plt.show()

print("\n--- Unique sets of Body Parts Tracked ---")
print(train_meta_df['body_parts_tracked'].value_counts())

print("\n--- Tracking Method Distribution ---")
print(train_meta_df['tracking_method'].value_counts())

In [ ]:
# --- Cell 4 (Corrected): Load Annotations and Add Metadata ---
import re

annotation_files = list(BASE_PATH.glob('train_annotation/*/*.parquet'))

all_annotations_list = []
for f in annotation_files:
    # Extract lab_id and video_id from the file path
    # The path is like: .../train_annotation/{lab_id}/{video_id}.parquet
    lab_id = f.parts[-2]
    video_id = int(f.stem) # f.stem gets the filename without extension

    # Load the data
    ann_df = pd.read_parquet(f)

    # Add the new columns
    ann_df['lab_id'] = lab_id
    ann_df['video_id'] = video_id

    all_annotations_list.append(ann_df)

all_annotations_df = pd.concat(all_annotations_list)

print("--- Annotations DataFrame with video_id and lab_id ---")
print(f"Total number of annotated events: {len(all_annotations_df)}")
display(all_annotations_df.head())

# The rest of the cell (plotting action counts) can remain the same.
action_counts = all_annotations_df['action'].value_counts()
print("\n--- Top 15 Most Frequent Behaviors ---")
print(action_counts.head(15))
# ... (plotting code) ...

plt.figure(figsize=(14, 8))
# Let's plot the top 20 for a better view
top_n = 20
sns.barplot(y=action_counts.index[:top_n], x=action_counts.values[:top_n], orient='h', palette='rocket')
plt.title(f'Top {top_n} Behavior Frequencies Across All Labs')
plt.xlabel('Number of Events')
plt.ylabel('Behavior')
plt.gca().invert_yaxis() # To show the most frequent at the top
plt.show()

In [ ]:
# --- Cell 5 (No changes needed now): Inspect a Single Tracking File ---
# This cell should now work without errors.
# Let's find a video that has annotations
annotated_video_ids = all_annotations_df['video_id'].unique()
sample_video_id = annotated_video_ids[0]

# We already have the lab_id in our merged annotations dataframe
sample_lab_id = all_annotations_df[all_annotations_df['video_id'] == sample_video_id].iloc[0]['lab_id']

tracking_file_path = BASE_PATH / f'train_tracking/{sample_lab_id}/{sample_video_id}.parquet'

sample_tracking_df = pd.read_parquet(tracking_file_path)

print(f"--- Exploring Video ID: {sample_video_id} from Lab: {sample_lab_id} ---")
print("Shape of tracking data:", sample_tracking_df.shape)
print("\nUnique mice:", sample_tracking_df['mouse_id'].unique())
print("Unique body parts:", sample_tracking_df['bodypart'].unique())
print("\nFirst 5 rows:")
display(sample_tracking_df.head())

# --- Cell 6 (No changes needed): Pivot Tracking Data ---
# This cell should also work now.
# ... (pivot function and call) ...

In [ ]:
# --- Cell 6: Pivot Tracking Data to Wide Format ---

def pivot_tracking_data(df):
    """Pivots the long-format tracking data to a wide format for a single video."""
    # First, get body parts on columns for each mouse
    pivoted_df = df.pivot_table(
        index=['video_frame', 'mouse_id'],
        columns='bodypart',
        values=['x', 'y']
    )
    # Flatten the multi-level column index (e.g., from ('x', 'nose') to 'x_nose')
    pivoted_df.columns = ['_'.join(col).strip() for col in pivoted_df.columns.values]
    pivoted_df = pivoted_df.reset_index()

    # Now, get each mouse's data onto the same row for each frame
    # We need to handle cases with more than 2 mice, but for now, let's focus on mouse1/mouse2
    # This assumes mouse_ids are like 'mouse1', 'mouse2'
    if all(pivoted_df['mouse_id'].isin(['mouse1', 'mouse2'])):
        final_df = pivoted_df.pivot(
            index='video_frame',
            columns='mouse_id',
        )
        # Flatten the multi-level columns again (e.g., from ('x_nose', 'mouse1') to 'x_nose_mouse1')
        final_df.columns = ['_'.join(col).strip() for col in final_df.columns.values]
        final_df = final_df.reset_index()
    else:
        # A more general approach if mouse_ids are not standard
        # This is more complex and we can defer it if not needed for the baseline
        print("Non-standard mouse IDs found. Returning intermediate pivot.")
        return pivoted_df

    return final_df

wide_df = pivot_tracking_data(sample_tracking_df)

print("--- Pivoted Wide Format ---")
print("Shape of wide data:", wide_df.shape)
display(wide_df.head())

In [ ]:
import os
import pandas as pd
import numpy as np
from pathlib import Path
import lightgbm as lgb
from tqdm import tqdm
import gc

# --- 1. Configuration ---
class CONFIG:
    BASE_PATH = Path('/kaggle/input/MABe-mouse-behavior-detection/')
    BEHAVIORS_TO_TRAIN = ['sniff', 'attack', 'rear', 'approach', 'selfgroom']
    LGB_PARAMS = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'n_estimators': 500,
        'learning_rate': 0.05,
        'num_leaves': 31,
        'max_depth': -1,
        'device': 'gpu',
        'gpu_platform_id': 0,
        'gpu_device_id': 0,
        'seed': 42,
        'n_jobs': -1,
        'verbose': -1,
        'gpu_use_dp': True,
        'max_bin': 63
    }
    PROB_THRESHOLD = 0.5
    MIN_FRAMES_FOR_EVENT = 3
    NEGATIVE_SAMPLING_RATIO = 4
    
    # 提交文件格式配置
    SUBMISSION_COLUMNS = ['row_id', 'video_id', 'agent_id', 'target_id', 'action', 'start_frame', 'stop_frame']
    REQUIRED_BEHAVIORS = ['sniff', 'attack', 'rear', 'approach', 'selfgroom']

# --- 2. Dynamic Data Loader ---
class DataLoader:
    def __init__(self, base_path):
        self.base_path = Path(base_path)
    
    def find_available_files(self):
        """动态查找可用的数据文件"""
        available_files = {}
        
        # 检查训练元数据文件
        train_files = ['train.csv', 'train_metadata.csv']
        for file in train_files:
            train_path = self.base_path / file
            if train_path.exists():
                available_files['train'] = train_path
                print(f"✅ 找到训练文件: {train_path}")
                break
        
        # 检查测试元数据文件
        test_files = ['test.csv', 'sample_submission.csv']
        for file in test_files:
            test_path = self.base_path / file
            if test_path.exists():
                available_files['test'] = test_path
                print(f"✅ 找到测试文件: {test_path}")
                break
        
        return available_files
    
    def load_metadata(self):
        """加载或推断元数据"""
        available_files = self.find_available_files()
        metadata = {}
        
        # 加载训练元数据
        if 'train' in available_files:
            metadata['train'] = pd.read_csv(available_files['train'])
        else:
            print("⚠️ 未找到训练元数据文件，从tracking文件推断...")
            metadata['train'] = self._infer_from_tracking('train')
        
        # 加载测试元数据
        if 'test' in available_files:
            test_df = pd.read_csv(available_files['test'])
            # 检查提交文件格式，确保列名正确
            if 'video_id' in test_df.columns:
                metadata['test'] = test_df
            else:
                print("⚠️ 测试文件格式不正确，从tracking文件推断...")
                metadata['test'] = self._infer_from_tracking('test')
        else:
            print("⚠️ 未找到测试元数据文件，从tracking文件推断...")
            metadata['test'] = self._infer_from_tracking('test')
        
        print(f"📊 训练数据: {len(metadata['train'])} 行")
        print(f"📊 测试数据: {len(metadata['test'])} 行")
        
        return metadata['train'], metadata['test']
    
    def _infer_from_tracking(self, data_type):
        """从tracking文件目录结构推断视频列表"""
        tracking_path = self.base_path / f'{data_type}_tracking'
        video_data = []
        
        if tracking_path.exists():
            print(f"🔍 从 {tracking_path} 推断视频列表...")
            lab_dirs = [d for d in tracking_path.iterdir() if d.is_dir()]
            
            for lab_dir in lab_dirs:
                lab_id = lab_dir.name
                video_files = list(lab_dir.glob('*.parquet'))
                
                for video_file in video_files:
                    try:
                        video_id = int(video_file.stem)
                        video_data.append({
                            'lab_id': lab_id,
                            'video_id': video_id
                        })
                    except ValueError:
                        continue
            
            if video_data:
                df = pd.DataFrame(video_data)
                print(f"🎯 推断出 {len(df)} 个{data_type}视频")
                return df
            else:
                print(f"❌ 无法从{data_type}_tracking推断视频列表")
                return pd.DataFrame(columns=['lab_id', 'video_id'])
        else:
            print(f"❌ {tracking_path} 不存在")
            return pd.DataFrame(columns=['lab_id', 'video_id'])

# --- 3. Submission Format Validator ---
class SubmissionValidator:
    def __init__(self, required_columns, required_behaviors):
        self.required_columns = required_columns
        self.required_behaviors = required_behaviors
    
    def validate_submission(self, submission_df):
        """验证提交文件格式"""
        print("🔍 验证提交文件格式...")
        
        # 检查列名
        missing_columns = set(self.required_columns) - set(submission_df.columns)
        if missing_columns:
            raise ValueError(f"缺少必要的列: {missing_columns}")
        
        # 检查列顺序
        if list(submission_df.columns) != self.required_columns:
            print("⚠️ 列顺序不正确，重新排序...")
            submission_df = submission_df[self.required_columns]
        
        # 检查空值
        if submission_df.isnull().any().any():
            print("⚠️ 发现空值，进行清理...")
            submission_df = submission_df.dropna()
        
        # 检查数据类型
        try:
            submission_df['row_id'] = submission_df['row_id'].astype(int)
            submission_df['video_id'] = submission_df['video_id'].astype(int)
            submission_df['agent_id'] = submission_df['agent_id'].astype(str)
            submission_df['target_id'] = submission_df['target_id'].astype(str)
            submission_df['action'] = submission_df['action'].astype(str)
            submission_df['start_frame'] = submission_df['start_frame'].astype(int)
            submission_df['stop_frame'] = submission_df['stop_frame'].astype(int)
        except Exception as e:
            raise ValueError(f"数据类型转换失败: {e}")
        
        # 检查行为名称
        invalid_actions = set(submission_df['action']) - set(self.required_behaviors)
        if invalid_actions:
            print(f"⚠️ 发现无效行为: {invalid_actions}，进行过滤...")
            submission_df = submission_df[submission_df['action'].isin(self.required_behaviors)]
        
        # 检查帧号有效性
        invalid_frames = submission_df[submission_df['start_frame'] > submission_df['stop_frame']]
        if not invalid_frames.empty:
            print("⚠️ 发现无效的帧范围，进行修正...")
            submission_df = submission_df[submission_df['start_frame'] <= submission_df['stop_frame']]
        
        # 检查帧号非负
        submission_df = submission_df[
            (submission_df['start_frame'] >= 0) & 
            (submission_df['stop_frame'] >= 0)
        ]
        
        print("✅ 提交文件格式验证通过")
        return submission_df.reset_index(drop=True)

# --- 4. GPU Support Check ---
def check_gpu_support():
    try:
        test_data = np.random.rand(100, 10).astype(np.float32)
        test_label = np.random.randint(0, 2, 100)
        params = {
            'objective': 'binary',
            'device': 'gpu',
            'verbose': -1
        }
        lgb.train(params, lgb.Dataset(test_data, label=test_label))
        print("✅ GPU加速可用")
        return True
    except Exception as e:
        print(f"⚠️ GPU加速不可用: {str(e)}")
        return False

# --- 5. Annotation Loader ---
def load_all_annotations(base_path):
    annotation_path = base_path / 'train_annotation'
    
    if not annotation_path.exists():
        print("❌ 标注路径不存在")
        return pd.DataFrame()
    
    annotation_files = list(annotation_path.glob('*/*.parquet'))
    if not annotation_files:
        print("❌ 未找到标注文件")
        return pd.DataFrame()
    
    all_annotations_list = []
    for f in tqdm(annotation_files, desc="加载标注文件"):
        try:
            lab_id = f.parts[-2]
            video_id = int(f.stem)
            ann_df = pd.read_parquet(f)
            ann_df['lab_id'] = lab_id
            ann_df['video_id'] = video_id
            all_annotations_list.append(ann_df)
        except Exception as e:
            print(f"⚠️ 加载标注文件失败 {f}: {e}")
            continue
    
    if all_annotations_list:
        return pd.concat(all_annotations_list).reset_index(drop=True)
    else:
        return pd.DataFrame()

# --- 6. Feature Engineering ---
CANONICAL_SKELETON = ['nose', 'ear_left', 'ear_right', 'neck', 'body_center', 'tail_base']

def process_video_data_lightweight(video_id, lab_id, tracking_path):
    try:
        file_path = tracking_path / f"{lab_id}/{video_id}.parquet"
        if not file_path.exists():
            return pd.DataFrame()
        
        tracking_df = pd.read_parquet(file_path)
        tracking_df['mouse_id'] = 'mouse' + tracking_df['mouse_id'].astype(str)
        
        cols_to_interpolate = ['x', 'y']
        tracking_df[cols_to_interpolate] = tracking_df.groupby(['mouse_id', 'bodypart'])[cols_to_interpolate].transform(
            lambda s: s.interpolate(method='linear', limit_direction='both')
        )
        
        wide_df = tracking_df.pivot_table(
            index='video_frame', 
            columns=['mouse_id', 'bodypart'], 
            values=['x', 'y']
        ).fillna(0)
        wide_df.columns = ['_'.join(map(str, col)) for col in wide_df.columns.values]
        
        features_list = []
        
        for mouse in ['mouse1', 'mouse2']:
            raw_coords = {}
            for canonical_part in CANONICAL_SKELETON:
                for potential_name in CANONICAL_SKELETON:
                    x_col = f'x_{mouse}_{potential_name}'
                    y_col = f'y_{mouse}_{potential_name}'
                    if x_col in wide_df.columns and y_col in wide_df.columns:
                        raw_coords[f'{mouse}_{canonical_part}_x'] = wide_df[x_col]
                        raw_coords[f'{mouse}_{canonical_part}_y'] = wide_df[y_col]
                        break
                else:
                    raw_coords[f'{mouse}_{canonical_part}_x'] = pd.Series(0, index=wide_df.index)
                    raw_coords[f'{mouse}_{canonical_part}_y'] = pd.Series(0, index=wide_df.index)
            
            center_x = raw_coords.get(f'{mouse}_body_center_x', 0)
            center_y = raw_coords.get(f'{mouse}_body_center_y', 0)
            
            for part in CANONICAL_SKELETON:
                x_norm = raw_coords.get(f'{mouse}_{part}_x', 0) - center_x
                y_norm = raw_coords.get(f'{mouse}_{part}_y', 0) - center_y
                x_norm.name = f'{mouse}_{part}_x_norm'
                y_norm.name = f'{mouse}_{part}_y_norm'
                features_list.extend([x_norm, y_norm])
        
        social_features_defs = [
            ('nose', 'nose'), ('nose', 'body_center'), 
            ('nose', 'tail_base'), ('body_center', 'body_center')
        ]
        
        for part1, part2 in social_features_defs:
            try:
                m1_x = wide_df[f'x_mouse1_{part1}']
                m1_y = wide_df[f'y_mouse1_{part1}']
                m2_x = wide_df[f'x_mouse2_{part2}']
                m2_y = wide_df[f'y_mouse2_{part2}']
                dist = np.sqrt((m1_x - m2_x)**2 + (m1_y - m2_y)**2)
                dist.name = f'dist_m1{part1}_m2{part2}'
                features_list.append(dist.fillna(0))
            except KeyError:
                continue
        
        if features_list:
            result = pd.concat(features_list, axis=1)
            for col in result.columns:
                if result[col].dtype == 'float64':
                    result[col] = result[col].astype('float32')
            return result
        else:
            return pd.DataFrame()
            
    except Exception as e:
        return pd.DataFrame()

# --- 7. Post-Processing ---
def probabilities_to_events(probs, threshold, min_frames, default_events=None):
    default_events = default_events or [(0, len(probs)-1)] if probs is not None else [(0, 99)]
    
    if probs is None or len(probs) == 0:
        return default_events
    
    binary_preds = (probs > threshold).astype(int)
    diffs = np.diff(binary_preds, prepend=0, append=0)
    starts = np.where(diffs == 1)[0]
    stops = np.where(diffs == -1)[0]
    
    events = [(s, e-1) for s, e in zip(starts, stops) if e-s >= min_frames]
    
    return events if events else default_events

# --- 8. Main Training and Inference Logic ---
if __name__ == "__main__":
    print("🚀 开始小鼠行为检测流程...")
    
    # 初始化组件
    data_loader = DataLoader(CONFIG.BASE_PATH)
    validator = SubmissionValidator(CONFIG.SUBMISSION_COLUMNS, CONFIG.REQUIRED_BEHAVIORS)
    
    # 检查GPU支持
    USE_GPU = check_gpu_support()
    if not USE_GPU:
        CONFIG.LGB_PARAMS['device'] = 'cpu'
        CONFIG.LGB_PARAMS['num_leaves'] = 63
        CONFIG.LGB_PARAMS['max_bin'] = 255
    
    # 动态加载元数据
    try:
        train_meta_df, test_meta_df = data_loader.load_metadata()
        if train_meta_df.empty or test_meta_df.empty:
            raise ValueError("无法加载元数据")
    except Exception as e:
        print(f"❌ 加载元数据失败: {e}")
        exit(1)
    
    # 加载标注数据
    all_annotations_df = load_all_annotations(CONFIG.BASE_PATH)
    if not all_annotations_df.empty:
        annotated_video_ids = all_annotations_df['video_id'].unique()
        train_meta_df_filtered = train_meta_df[train_meta_df['video_id'].isin(annotated_video_ids)].copy()
    else:
        train_meta_df_filtered = train_meta_df.copy()
    
    # 训练模型
    models = {}
    
    for behavior in CONFIG.BEHAVIORS_TO_TRAIN:
        print(f"\n--- 处理行为: {behavior} ---")
        X_train_list, y_train_list = [], []
        
        for row in tqdm(train_meta_df_filtered.itertuples(), total=len(train_meta_df_filtered), desc=f"处理视频"):
            try:
                features_df = process_video_data_lightweight(
                    row.video_id, row.lab_id, CONFIG.BASE_PATH / 'train_tracking'
                )
                if features_df.empty:
                    continue
                    
                labels = pd.Series(np.zeros(len(features_df)), name=behavior)
                
                if not all_annotations_df.empty:
                    video_ann = all_annotations_df[
                        (all_annotations_df['video_id'] == row.video_id) & 
                        (all_annotations_df['action'] == behavior)
                    ]
                    for _, ann_row in video_ann.iterrows():
                        start, stop = ann_row['start_frame'], ann_row['stop_frame']
                        if start < len(labels):
                            labels.iloc[start:min(stop + 1, len(labels))] = 1
                
                positive_indices = labels[labels == 1].index
                if len(positive_indices) == 0:
                    continue
                
                negative_indices = labels[labels == 0].index
                num_neg_to_sample = min(
                    int(len(positive_indices) * CONFIG.NEGATIVE_SAMPLING_RATIO), 
                    len(negative_indices)
                )
                sampled_negative_indices = np.random.choice(
                    negative_indices, size=num_neg_to_sample, replace=False
                )
                final_indices = np.concatenate([positive_indices, sampled_negative_indices])
                np.random.shuffle(final_indices)
                
                X_train_list.append(features_df.iloc[final_indices])
                y_train_list.append(labels.iloc[final_indices])
                
            except Exception as e:
                continue
        
        if not X_train_list:
            print(f"❌ 没有{behavior}的训练数据，跳过")
            continue
        
        X_train = pd.concat(X_train_list).reset_index(drop=True)
        y_train = pd.concat(y_train_list).reset_index(drop=True)
        
        print(f"📊 训练数据形状: X={X_train.shape}, y={y_train.shape}")
        
        try:
            model = lgb.LGBMClassifier(**CONFIG.LGB_PARAMS)
            model.fit(X_train, y_train)
            models[behavior] = model
            print(f"✅ {behavior}模型训练完成")
        except Exception as e:
            print(f"❌ {behavior}模型训练失败: {e}")
            continue
        
        del X_train, y_train, X_train_list, y_train_list
        gc.collect()
    
    # 推理预测
    print("\n--- 开始测试集推理 ---")
    all_predictions = []
    
    if not models:
        print("❌ 没有训练成功的模型")
    else:
        for row in tqdm(test_meta_df.itertuples(), total=len(test_meta_df), desc="推理测试视频"):
            try:
                features_df = process_video_data_lightweight(
                    row.video_id, row.lab_id, CONFIG.BASE_PATH / 'test_tracking'
                )
                if features_df.empty:
                    continue
                
                for behavior, model in models.items():
                    probs = model.predict_proba(features_df)[:, 1]
                    events = probabilities_to_events(probs, CONFIG.PROB_THRESHOLD, CONFIG.MIN_FRAMES_FOR_EVENT)
                    
                    for start, stop in events:
                        agent_id = 'mouse1'
                        target_id = 'mouse2' if behavior not in ['selfgroom', 'rear'] else 'mouse1'
                        all_predictions.append({
                            'video_id': int(row.video_id), 
                            'agent_id': str(agent_id), 
                            'target_id': str(target_id), 
                            'action': str(behavior), 
                            'start_frame': int(start), 
                            'stop_frame': int(stop)
                        })
                        
            except Exception as e:
                continue
    
    # 生成并验证提交文件
    print("\n📝 生成提交文件...")
    
    if all_predictions:
        submission_df = pd.DataFrame(all_predictions)
        submission_df['row_id'] = range(len(submission_df))
    else:
        # 如果没有预测，创建空的但格式正确的DataFrame
        submission_df = pd.DataFrame(columns=CONFIG.SUBMISSION_COLUMNS)
        submission_df['row_id'] = []
        submission_df['video_id'] = []
        submission_df['agent_id'] = []
        submission_df['target_id'] = []
        submission_df['action'] = []
        submission_df['start_frame'] = []
        submission_df['stop_frame'] = []
    
    # 验证和清理提交文件
    try:
        submission_df = validator.validate_submission(submission_df)
    except Exception as e:
        print(f"❌ 提交文件验证失败: {e}")
        # 创建最小有效提交文件
        submission_df = pd.DataFrame(columns=CONFIG.SUBMISSION_COLUMNS)
        submission_df['row_id'] = [0]
        submission_df['video_id'] = [0]
        submission_df['agent_id'] = ['mouse1']
        submission_df['target_id'] = ['mouse2']
        submission_df['action'] = ['sniff']
        submission_df['start_frame'] = [0]
        submission_df['stop_frame'] = [1]
    
    # 确保列顺序正确
    submission_df = submission_df[CONFIG.SUBMISSION_COLUMNS]
    
    # 最终检查
    print("🔍 最终格式检查...")
    print(f"列名: {list(submission_df.columns)}")
    print(f"数据类型:")
    for col in submission_df.columns:
        print(f"  {col}: {submission_df[col].dtype}")
    print(f"数据形状: {submission_df.shape}")
    print(f"空值检查: {submission_df.isnull().sum().sum()}")
    
    # 保存提交文件
    submission_df.to_csv('submission.csv', index=False)
    print(f"✅ 提交文件保存成功，共 {len(submission_df)} 行")
    
    # 显示前几行作为参考
    if not submission_df.empty:
        print("\n📋 提交文件前5行:")
        print(submission_df.head())
    else:
        print("⚠️ 提交文件为空，但格式正确")
    
    print("🎉 流程完成！")